In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, Conv3D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling3D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import regularizers
from sklearn.metrics import f1_score
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image 
import seaborn as sns
import os
import re
import glob
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn import preprocessing
import tqdm
from numpy import loadtxt
from sklearn.utils import class_weight

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [3]:
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap=plt.cm.Blues) #'viridis'
    #plt.savefig('/home/jovyan/Confmatrix.png')
    plt.show()
    
    return conf_matx

In [4]:
def plot_history(model_history, model_name):
    fig = plt.figure(figsize=(15,5), facecolor='w')
    ax = fig.add_subplot(121)
    ax.plot(model_history.history['loss'])
    ax.plot(model_history.history['val_loss'])
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    ax = fig.add_subplot(122)
    ax.plot(model_history.history['accuracy'])
    ax.plot(model_history.history['val_accuracy'])
    ax.set(title=model_name + ': Model Accuracy; test='+ str(np.round(model_history.history['val_accuracy'][-1], 3)),
           ylabel='Accuracy', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    #plt.savefig('/home/jovyan/Acc_loss.png')
    plt.show()
   
    
    return fig

In [5]:
def loadImages(path_data, path_labels):

    image_list = []
    

    for filename in tqdm.tqdm(sorted(glob.glob(path_data), key=natural_keys)): 
        im=cv2.imread(filename)
        #gray_image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        imarray = np.array(im)
        #imarray = imarray.astype('float32')
        image_list.append(imarray)

    x_orig = np.reshape(image_list, (len(image_list), 90, 90, 3))
    
    path = path_labels    
    labels = pd.read_csv(path, usecols=["Type", "Category"],
                       sep="," )
    y_orig = np.array(labels['Category'])

    return x_orig, y_orig

In [6]:
p_data = '/home/jovyan/DATA_MASTER_PROJECT/Check_DIFF_T60_T_97/3_class_Cont_ADR_HRH/'

train_data =  p_data + 'TRAIN/*.tiff'
train_lab= p_data + 'train_lab.csv'

validation_data = p_data + 'VALIDATION/*.tiff'
validation_lab = p_data + 'validation_lab.csv'


test_data= p_data + 'TEST/*.tiff'
test_lab= p_data + 'test_lab.csv'



In [21]:
def resize(x):
    rescaled = []

    for i in x:

        scale_percent = 140 # percent of original size
        width = int(i.shape[1] / (scale_percent / 100))
        height = int(i.shape[0] / (scale_percent / 100))
        dim = (width, height)
        resized = cv2.resize(i, dim, interpolation = cv2.INTER_LANCZOS4)
        rescaled.append(resized)

    x_orig = np.reshape( rescaled, (len( rescaled), resized.shape[1], resized.shape[1], 3))

    return x_orig

In [8]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=2)

In [9]:
pretrained_model = VGG16(weights='imagenet',include_top=False, input_shape=(64, 64, 3))

base_model = Model(inputs=pretrained_model.input, outputs=pretrained_model.get_layer('block3_pool').output)

In [10]:
x_train, y_train = loadImages(train_data, train_lab)
x_train.shape, y_train.shape

100%|██████████| 73354/73354 [01:06<00:00, 1109.69it/s]


((73354, 90, 90, 3), (73354,))

In [11]:
x_val, y_val = loadImages(validation_data, validation_lab)
x_val.shape, y_val.shape

100%|██████████| 19653/19653 [00:19<00:00, 1028.93it/s]


((19653, 90, 90, 3), (19653,))

In [12]:
x_test, y_test = loadImages(test_data, test_lab)
x_test.shape, y_test.shape

100%|██████████| 24106/24106 [00:23<00:00, 1031.18it/s]


((24106, 90, 90, 3), (24106,))

In [22]:
x_train = resize(x_train)
x_train.shape

(73354, 64, 64, 3)

In [23]:
x_val = resize(x_val)
x_val.shape

(19653, 64, 64, 3)

In [24]:
x_test = resize(x_test)
x_test.shape

(24106, 64, 64, 3)

In [13]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 26564, 1: 36776, 2: 10014}

In [14]:
unique, counts = np.unique(y_val, return_counts=True)
dict(zip(unique, counts))

{0: 7062, 1: 9861, 2: 2730}

In [15]:
weights = class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)
print('weights = ' + str(weights))

weights = [0.9204688  0.66487202 2.44171493]


In [16]:
y_train = keras.utils.to_categorical(y_train)
y_test_1 = keras.utils.to_categorical(y_test)
y_val = keras.utils.to_categorical(y_val)

y_train.shape, y_test_1.shape, y_val.shape

((73354, 3), (24106, 3), (19653, 3))

In [17]:
x_train = preprocess_input(x_train)

x_val = preprocess_input(x_val)

x_test = preprocess_input(x_test)


In [27]:
res=[]

op = [Adam, RMSprop]
n_op = ['Adma', 'RMSP']

batch_size = [64,32]
learn_r = [1e-4, 1e-5]
act = ['relu', 'tanh' ]
comb = []
for a in tqdm.tqdm(act):
    for b in batch_size:
        for i in learn_r:
            for j in learn_r:
                for o, n in zip(op, n_op):

                    comb.append(list((i,j,a,b, n)))

                    dat_val = ImageDataGenerator()

                    dat_val.fit(x_val)

                    val_gen = dat_val.flow(x_val, y_val, batch_size=128)

                    datagen = ImageDataGenerator()

                    datagen.fit(x_train)

                    train_gen = datagen.flow(x_train, y_train, batch_size=128)


                    m4 = Sequential()
                    m4.add(base_model)


                    m4.add(BatchNormalization())
                    m4.add(GlobalAveragePooling2D())
                    m4.add(Dense(b, activation= a))
                    m4.add(Dense(3, activation = 'softmax'))






                    base_model.trainable = False

                    opt = o(lr=i)

                    m4.compile(loss= keras.losses.categorical_crossentropy, optimizer=opt, metrics = ['accuracy'])



                    epochs = 100

                    m4_h = m4.fit(train_gen,
                                    steps_per_epoch=(len(x_train)/128),
                                    callbacks = [es],
                                    epochs=epochs,
                                    validation_data = val_gen, 
                                    validation_steps = (len(x_val)/128),
                                    class_weight = weights, verbose=0)

                    base_model.trainable = True

                    opt = o(lr=j)

                    m4.compile(loss= keras.losses.categorical_crossentropy, optimizer=opt, metrics = ['accuracy'])


                    epochs = 100

                    m4_h = m4.fit(train_gen,
                                    steps_per_epoch=(len(x_train)/128),
                                    callbacks = [es],
                                    epochs=epochs,
                                    validation_data = val_gen, 
                                    validation_steps = (len(x_val)/128),
                                    class_weight = weights,
                                    verbose=0)

                    #plot_history(m4_h, 'first')


                    scores = m4.evaluate(x_test, y_test_1)
                    #print("\n%s: %.2f%%" % (m4.metrics_names[1], scores[1]*100))
                    res.append(scores[1]*100)
                    
                    del m4
                    K.clear_session()

#test_preds = m4.predict(x_test)



  0%|          | 0/2 [00:00<?, ?it/s]

  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 6s 230us/sample - loss: 1.3444 - accuracy: 0.6052
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 5s 223us/sample - loss: 1.5608 - accuracy: 0.6036
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 6s 233us/sample - loss: 1.1637 - accuracy: 0.6353
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 6s 229us/sample - loss: 1.4769 - accuracy: 0.6340
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 6s 232us/sample - loss: 1.7739 - accuracy: 0.5792
  ...
    to  




 50%|█████     | 1/2 [59:02<59:02, 3542.62s/it]

  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 6s 229us/sample - loss: 1.7317 - accuracy: 0.6288
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 6s 229us/sample - loss: 2.3767 - accuracy: 0.6061
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 5s 227us/sample - loss: 1.3607 - accuracy: 0.6483
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 6s 229us/sample - loss: 2.4180 - accuracy: 0.6464
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
24106/24106 [==============================] - 5s 227us/sample - loss: 1.4928 - accuracy: 0.6430
  ...
    to  




100%|██████████| 2/2 [1:52:08<00:00, 3364.42s/it]


In [28]:
result = list(zip(res, comb))


In [29]:
sorted(result, reverse=True)

[(64.88426327705383, [1e-05, 1e-05, 'relu', 64, 'RMSP']),
 (64.86766934394836, [1e-05, 1e-05, 'tanh', 64, 'Adma']),
 (64.86766934394836, [1e-05, 1e-05, 'relu', 64, 'Adma']),
 (64.8386299610138, [1e-05, 1e-05, 'tanh', 64, 'RMSP']),
 (64.83033299446106, [0.0001, 1e-05, 'tanh', 64, 'Adma']),
 (64.77225422859192, [1e-05, 1e-05, 'relu', 32, 'RMSP']),
 (64.77225422859192, [1e-05, 1e-05, 'relu', 32, 'Adma']),
 (64.64365720748901, [0.0001, 1e-05, 'tanh', 64, 'RMSP']),
 (64.52750563621521, [1e-05, 0.0001, 'relu', 32, 'Adma']),
 (64.45698142051697, [0.0001, 1e-05, 'tanh', 32, 'Adma']),
 (64.43209052085876, [0.0001, 1e-05, 'relu', 32, 'RMSP']),
 (64.4030511379242, [0.0001, 1e-05, 'relu', 32, 'Adma']),
 (64.4030511379242, [1e-05, 1e-05, 'tanh', 32, 'Adma']),
 (64.29934501647949, [1e-05, 0.0001, 'tanh', 64, 'Adma']),
 (64.16245102882385, [0.0001, 1e-05, 'tanh', 32, 'RMSP']),
 (64.12926316261292, [1e-05, 1e-05, 'tanh', 32, 'RMSP']),
 (63.614869117736816, [1e-05, 0.0001, 'tanh', 64, 'RMSP']),
 (63.57

In [ ]:
preds_df = pd.DataFrame(test_preds)
predicted_labels = preds_df.idxmax(axis=1)

In [ ]:
draw_confusion_matrix(y_test, predicted_labels)

In [ ]:
print(classification_report(y_test, predicted_labels))

In [ ]:
p = '/home/jovyan/Saved_Models/'
m4.save(p + "CNN.h5")